<a href="https://colab.research.google.com/github/Janani-R18/NM-assignments/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import gradio as gr
from datetime import date

def predict_stock_price(stock_symbol, start_date, end_date, predict_date):
    # Fetch historical data
    try:
        stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
    except:
        return "Invalid stock symbol or date range", None

    if len(stock_data) < 5:
        return "Insufficient historical data", None

    # Prepare data for linear regression
    df = stock_data[['Close']].reset_index()
    df['Date_ordinal'] = df['Date'].map(date.toordinal)

    # Create features and target
    X = df[['Date_ordinal']]
    y = df['Close']

    # Train model
    model = LinearRegression()
    model.fit(X, y)

    # Prepare prediction input
    predict_ordinal = date.toordinal(predict_date)
    prediction = model.predict([[predict_ordinal]])[0]

    # Create visualization data
    df['Predicted'] = model.predict(X)
    plot_data = df[['Date', 'Close', 'Predicted']].melt(id_vars='Date')

    return f"Predicted price for {predict_date}: ${prediction:.2f}", plot_data

# Gradio interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📈 AI Stock Price Predictor")

    with gr.Row():
        with gr.Column():
            symbol = gr.Textbox(label="Stock Symbol (e.g., AAPL)", value="AAPL")
            # Use gr.components.DatePicker instead of gr.DatePicker
            start_date = gr.DatePicker(label="Training Start Date", value=date(2020, 1, 1))
            end_date = gr.DatePicker(label="Training End Date", value=date(2023, 1, 1))
            predict_date = gr.DatePicker(label="Prediction Date", value=date(2025, 12, 31))
            submit_btn = gr.Button("Predict", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Prediction Result")
            plot = gr.LinePlot(
                x="Date",
                y="value",
                color="variable",
                title="Historical vs Predicted Prices",
                overlay_point=True,
                width=600,
                height=400
            )

    submit_btn.click(
        fn=predict_stock_price,
        inputs=[symbol, start_date, end_date, predict_date],
        outputs=[output_text, plot]
    )

demo.launch()


AttributeError: module 'gradio' has no attribute 'DatePicker'